<a href="https://colab.research.google.com/github/dowrave/PythonToKaggle/blob/main/2_NLP_Twitter_Disaster_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file"{name}" with length {length} bytes'.format(name = fn, length = len(uploaded[fn])))

!mkdir -p ~/.kaggle/
!mv kaggle.json ~/.kaggle/ 
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c nlp-getting-started

In [ ]:
!unzip nlp-getting-started.zip

In [ ]:
# 용량 확인하는 코드
import os
DATA_PATH = "./"
for file in os.listdir(DATA_PATH):
  if 'csv' in file and 'zip' not in file:
    print(file.ljust(30) + str(round(os.path.getsize(file)/ 1000000, 2)) + 'MB')

In [ ]:
import pandas as pd
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

## 탐색적 자료 분석
- 분류 문제에서 가장 중요한 건 종속 변수를 시각화해서 분포를 확인하는 것이다
  - 종속 변수(Y)는 보통 비대칭인 경우가 더 많기 때문에 이러한 비대칭 데이터를 어떻게 샘플링하여 학습시킬 것인지가 핵심임

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
news_class = train['target'].value_counts()
labels = ['Non-Disaster', 'Disaster']

fig, ax =  plt.subplots(figsize = (10, 6))
ax.bar(labels, news_class, color = ['green', 'orange'])

fig.show()

In [ ]:
disaster_tweet_len = train[train['target'] == 1]['text'].str.len()
non_disaster_tweet_len = train[train['target'] == 0]['text'].str.len()

disaster_tweet_len # 단어 수가 아니라 글자 수임

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].hist(disaster_tweet_len, color = 'green')
ax[0].set_title('Disaster Tweet Length')

ax[1].hist(non_disaster_tweet_len, color = 'orange')
ax[1].set_title('Non Diaster Tweet Length')

fig.suptitle('All words in Tweets')
fig.show()

In [ ]:
# 두 그래프의 분포가 비슷한 모양이지만(?) 140자가 넘을 때 Non-Disaster의 글자 수가 급감하는 경향이 있음
# 이를 Boxplot으로 확인해보자
fig,ax = plt.subplots(1, 2, figsize = (12, 6))
ax[0].boxplot(disaster_tweet_len, labels=['counts'], 
              showmeans = True) # 평균값을 그래프에 표시함
ax[0].set_title("Disaster Tweet Length")

ax[1].boxplot(non_disaster_tweet_len, labels=['counts'], 
              showmeans = True) # 평균값을 그래프에 표시함
ax[1].set_title("Non Disaster Tweet Length")

fig.suptitle('All words in Tweets')
plt.show()

In [ ]:
print(disaster_tweet_len.describe(), '\n', '-'*20, '\n', non_disaster_tweet_len.describe())


In [ ]:
# WordCLoud로 사용된 데이터의 빈도수를 확인할 수 있다.
from wordcloud import WordCloud, STOPWORDS

disaster_tweet_keywords = dict(train[train['target'] == 1]['keyword'].value_counts())
non_disaster_tweet_keywords = dict(train[train['target'] == 0]['keyword'].value_counts())


stopwords = set(STOPWORDS)
disaster_wordcloud = WordCloud(stopwords = stopwords,
                               width = 800, height = 400,
                               background_color = 'white').generate_from_frequencies(disaster_tweet_keywords)
non_disaster_wordcloud = WordCloud(stopwords = stopwords,
                               width = 800, height = 400,
                               background_color = 'white').generate_from_frequencies(non_disaster_tweet_keywords)

fig, ax = plt.subplots(1, 2, figsize=(16, 10))
ax[0].imshow(disaster_wordcloud, interpolation = 'bilinear')
ax[0].axis('off')
ax[0].set_title('Disaster Tweet')
ax[1].imshow(non_disaster_wordcloud, interpolation = 'bilinear')
ax[1].axis('off')
ax[1].set_title('Non Disaster Tweet')

fig.show()

### 와! 영어시간!
- derailment : 탈선
- wreckage : 난파 
- 재난 트윗은 '명사'가 주로 쓰임
- 비재난 트윗은 형용사나 동사가 주로 쓰임

## Feature Engineering

### 1. 결측치 확인

In [ ]:
def check_na(data):
  isnull_na = (data.isnull().sum() / len(data)) * 100
  data_na = isnull_na.drop(isnull_na[isnull_na == 0].index).sort_values(ascending = False)
  missing_data = pd.DataFrame({'Missing Ratio' : data_na,
                               'Data Type' : data.dtypes[data_na.index]})
  print('결측치 데이터 칼럼과 건수 : \n', missing_data)

check_na(train)
check_na(test)

In [ ]:
# 이용할 데이터는 'text' 밖에 없기 때문에 나머지는 모두 제거하고, test_id만 따로 저장해둔다
test_id = test['id']
for datas in [train, test]:
  datas = datas.drop(['id', 'keyword', 'location'], axis = 1, 
                     inplace = True)
train.shape, test.shape

### 텍스트 전처리 함수 만들기
- 기본적으로 웹 크롤링으로 데이터를 불러옴(실무 종사자가 아니라면)
- 처리 과정엔 이런 것들이 있다
  - URL 문자 삭제, HTML 태그 삭제, 이모티콘 삭제, 특수 문자 공백화, 구두점 삭제, 대문자 -> 소문자 변환, 불용어 제거

In [ ]:
import string
import re
# NLP에 가장 많이 쓰이는 파이썬 라이브러리 : NLTK, Spacy
from nltk.corpus import stopwords # NLTK에는 다양한 Corpus(말뭉치)가 존재하며, 그 중 불용어만 다운
import nltk
nltk.download('stopwords')

In [ ]:
def data_cleansing(text, remove_stopwords = False):

  # remove url
  url = re.compile(r'https?//\S+|www\.\S+')
  """ 정규식 설명
  '' 밖의 r : raw string - 탈출 문자(\)의 작동을 막음.
    - 뒤에 www\. 여기의 .을 0문자 이상 매칭이 아니라 . 그 자체로 쓰기 위해 씀
  ? - 0 or 1 : 즉 http or https 둘 다 매칭
  \S : Whitespace가 아닌 문자와 매칭 : \s는 whitespace와 매칭
    - 이건 탈출문자로 쓰인 게 아니다. 그래서 멀쩡히 잘 작동함
  """
  cleaned_text = url.sub(r'', text) # compile 내부 정규식을 이용해 text를 ''로 바꾸겠다
                                    # 알고 있듯이 compile을 이용하지 않고 sub 내부에서 바로 해결할 수도 있다.

  # remove html
  html = re.compile(r'<.*?>')
  """
  꺾쇠(<, >)에는 특별한 의미가 있지 않음. html 태그는 <> 내부에 뭐가 들어가니까 쓴 거
  . : \n을 제외한 모든 문자 매칭
  * : 0문자 이상의 '반복' 매칭 
  ? : Greedy한 반복 매칭 문자를 Reluctant하게 바꿔준다. 가장 적은 반복에 대한 매칭을 찾아준다.
  """
  cleaned_text = html.sub(r'', cleaned_text)

  # remove emoji
  emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F" # 이모티콘
                            u"\U0001F300-\U0001F5FF" # symbol & pictograph
                            u"\U0001F680-\U0001F6FF" # transport & map symbol
                            u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            "]+", flags = re.UNICODE)
  cleaned_text = emoji_pattern.sub(r'', cleaned_text)

  # special letters to empty space
  cleaned_text = re.sub("[^a-zA-Z]", "", cleaned_text)

  # Remove Punctuation
  table = str.maketrans('', '', string.punctuation)
  cleaned_text = cleaned_text.translate(table)

  # Lowercase
  cleaned_text = cleaned_text.lower().split()

  if remove_stopwords:
    stops = set(stopwords.words("english"))
    cleaned_text = [word for word in cleaned_text if not word in stops]
    clean_review = ' '.join(cleaned_text)
  else:
    clean_review = ' '.join(cleaned_text)

  return clean_review



In [ ]:
clean_train_reviews = []
for datas in [train, test]:
  datas['cleaned_text'] = datas['text'].apply(lambda x: data_cleansing(x, remove_stopwords = True))

train.head()

In [ ]:
# 불용어 확인
print(len(stopwords.words('english')))
print(stopwords.words('english')[:10])

## 특징 추출하기
- 단어, 문장들을 개별적인 값으로 바꾸는 게 매우매우 중요하다
- 방법으로는 2가지가 있다


  1. CountVectorizer
    - 텍스트 데이터가 단순히 몇 번 나왔는가


In [ ]:
# 예제 
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['As you know, I want to be with you',
          'Thank you, but I cannot be with you']
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 각 단어의 해당 문장 내에서의 등장 빈도, 인덱스는 아래를 따름
print(vector.vocabulary_) # 위 리스트의 각 인덱스의 의미. 

- 단점 : 빈도만이 중요하다면, 대명사 등은 중요한 단어로 취급될 것이다.
  - 위의 불용어 리스트에서도 보이듯이, 대명사는 제거 대상이다. 즉 빈도가 잦더라도 중요성을 담고 있지 못함을 의미한다

2. TfidfVectorizer
    - TF-IDF를 따른다. (이 두 값을 곱함)
      - TF(Term Frequency) : 단어의 데이터 '내'에서의 등장빈도
      - IDF(Inverse Document Frequency) : 특정 단어의 여러 문서에서의 등장 빈도(DF).. 의 역수
        - 즉 어떤 단어가 여러 문서에 걸쳐 고루 나타난다면 이는 중요하지 않다는 의미가 된다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
corpus = ['Can I have lunch with you?', 
          'No, I cannot have it with you',
          'Because, I need to study later']
tfidfv = TfidfVectorizer().fit(corpus)
print(np.round(tfidfv.transform(corpus).toarray(), 2)) # 각 단어의 "점수"가 표현됨. 등장하지 않았다면 0
print(tfidfv.vocabulary_)

- TF-IDF의 단점 
  - 희소 행렬(Sparse Matrix : 행렬 구조 내에 0이 겁나게 많은 거)의 발생으로 인한 저장 공간의 낭비 
  - ML 연산 학습에서의 메모리 낭비

In [ ]:
# 이번 예제에서는 TfidfVectorizer를 씁니다
vectorizer = TfidfVectorizer(min_df = 0.0, # 디폴트 1 : 문서에서 이 값보다 등장빈도가 적다면 취급 안함(Threshold 개념)
                             analyzer = 'char', # word, char 등이 올 수 있고, character N-gram을 의미함 (???)
                             sublinear_tf = True, # tf값을 1 + log(tf) 값으로 바꿈
                             ngram_range = (1, 3), # Uni, BI, Trigram까지 이용하겠다
                             max_features = 10000) # Corpus 내에서 상위 빈도 10000개만 뽑음
X = vectorizer.fit_transform(train['cleaned_text']).todense()
y = train['target'].values
print(X.shape, y.shape) # (학습할 데이터 숫자, 현재 데이터에 사용되는 전체 단어의 개수)

## ML 모델 학습 및 평가

### 1. 로지스틱 회귀 모델
- 분류에서 제일 많이 등장하는 '초기 모델'
- 초기 값은 0.5로 설정, 임계값을 설정해 1에 가까우면 1, 0에 가까우면 0으로 표시한다

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3,
                                                      random_state = 0)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
lgs = LogisticRegression(class_weight = 'balanced') # 데이터가 비대칭임을 반영하는 파라미터
lgs.fit(X_train, y_train)

In [ ]:
# 바로 제출까지
X_testset = vectorizer.transform(test['cleaned_text']).todense() # 희소행렬 변환(todense)
print(X_testset.shape)

In [ ]:
y_test_pred = lgs.predict(X_testset)
print(y_test_pred)
y_test_pred = np.where(y_test_pred>=0.5, 1, 0)
print(y_test_pred)

submission_file = pd.DataFrame({'id' : test_id, 'target' : y_test_pred})
submission_file.to_csv('submission_lgs.csv', index = False)

### ML 모델의 평가

- 위랑 직접적인 관련이 있는 건 아니지만..



- Confusion Matrix, Accuracy, Precision, Recall 등
- 최적의 Threshold 값 : G-Mean
  - https://towardsdatascience.com/optimal-threshold-for-imbalanced-classification-5884e870c293
- ROC Curve, AUC

In [ ]:
# 함수 만드는 중.. 아마 numpy로 하면 훨씬 빠를걸? 싶지만 그냥 연습용이다.
y_true = [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]
# y_pred = [0.1, 0.3, 0.2, .6, .8, .05, .9, .5, .3, .66, .3, .2, .85, .15, .99]
thresholds = [0, .1, .2, .3, .4, .5, .6, .7, .8, .85, .9, .99, 1.0]
cf = {"tp" : 0,
      "fn" : 0,
      "tn" : 0,
      "fp" : 0}
tpr_list = []
fpr_list = []
auc_list = []


for th in thresholds:
  y_pred = [0.1, 0.3, 0.2, .6, .8, .05, .9, .5, .3, .66, .3, .2, .85, .15, .99]
  for idx, val in enumerate(y_pred):
    if val < th:
      y_pred[idx] = 0
    else:
      y_pred[idx] = 1
  print(y_pred)

  for lst in zip(y_true, y_pred):
    if (lst[0] == 0) & (lst[1] == 0):
      cf['tn'] += 1
    elif (lst[0] == 1) & (lst[1] == 0):
      cf['fn'] += 1
    elif (lst[0] == 0) & (lst[1] == 1):
      cf['fp'] += 1
    else:
      cf['tp'] += 1

  print(cf)

  tpr = cf['tp'] / (cf['tp'] + cf['fn'])
  fpr = cf['fp'] / (cf['fp'] + cf['tn'])

  tpr_list.append(tpr)
  fpr_list.append(fpr)

  # return tpr_list, fpr_li

In [ ]:
""" 이 책은 이런 지점이 아쉽다.. 
tpr이나 fpr 함수를 찾을 수가 없음. 빵꾸가 많다.
그러면 만들어야지 ㅋㅋ; """
print(pd.DataFrame({"Thresholds" : thresholds,
                    "fpr" : fpr_list,
                    "tpr" : tpr_list}))

fig, ax = plt.subplots(figsize=(10, 6))
ax.fill_between(fpr_list, tpr_list, alpha = 0.4)
ax.plot(fpr_list, tpr_list, lw = 2, label = 'ROC')
plt.plot([0, 1], [0, 1], lw = 2, color = 'r', label = 'Random')
ax.set_xlim(0, 1.0)
ax.set_ylim(0, 1.0)
ax.set_xlabel("FPR", fontsize = 15)
ax.set_ylabel("TPR", fontsize = 15)
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

y_true = [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]
y_pred = [.1, .3, .2, .6, .8, .05, .9, .5, .3, .66, .3, .2, .85, .15, .99]

print(roc_auc_score(y_true, y_pred))